In [1]:
#r "nuget:BenchmarkDotNet"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages BenchmarkDotNet, 0.13.7

In [2]:
using System;
using System.Linq.Expressions;
using System.Reflection;
using System.Reflection.Emit;
using BenchmarkDotNet.Attributes;
using BenchmarkDotNet.Running;

public class SampleClass
{
    public void PrintMessage()
    {
        // Do nothing for the sake of the benchmark
    }
}

public delegate void DynamicMethodDelegate(object target);

[MemoryDiagnoser] // This attribute reports memory allocations
public class InvokeBenchmark
{
    private readonly SampleClass sampleInstance = new SampleClass();
    private readonly MethodInfo methodInfo;
    private readonly DynamicMethodDelegate dynamicDelegate;
    private readonly Action<SampleClass> expressionDelegate;

    public InvokeBenchmark()
    {
        methodInfo = typeof(SampleClass).GetMethod("PrintMessage");

        // Reflection.Emit
        var dynamicMethod = new DynamicMethod("DynamicInvoke", typeof(void), new Type[] { typeof(object) }, typeof(SampleClass));
        var ilGen = dynamicMethod.GetILGenerator();
        ilGen.Emit(OpCodes.Ldarg_0);
        ilGen.Emit(OpCodes.Castclass, typeof(SampleClass));
        ilGen.Emit(OpCodes.Callvirt, methodInfo);
        ilGen.Emit(OpCodes.Ret);
        dynamicDelegate = (DynamicMethodDelegate)dynamicMethod.CreateDelegate(typeof(DynamicMethodDelegate));

        // Expressions
        var parameter = Expression.Parameter(typeof(SampleClass), "obj");
        var methodCall = Expression.Call(parameter, methodInfo);
        var lambda = Expression.Lambda<Action<SampleClass>>(methodCall, parameter);
        expressionDelegate = lambda.Compile();
    }

    [Benchmark]
    public void MethodInfoInvoke()
    {
        methodInfo.Invoke(sampleInstance, null);
    }

    [Benchmark]
    public void DynamicDelegateInvoke()
    {
        dynamicDelegate(sampleInstance);
    }

    [Benchmark]
    public void ExpressionDelegateInvoke()
    {
        expressionDelegate(sampleInstance);
    }
}

In [3]:
BenchmarkRunner.Run<InvokeBenchmark>();

]9;4;3;0\]9;4;0;0\

Error: System.UnauthorizedAccessException: Access to the path 'c:\Program Files\Microsoft VS Code\BenchmarkDotNet.Artifacts\results' is denied.
   at System.IO.FileSystem.CreateDirectory(String fullPath, Byte[] securityDescriptor)
   at System.IO.Directory.CreateDirectory(String path)
   at BenchmarkDotNet.Extensions.CommonExtensions.CreateIfNotExists(String directoryPath)
   at BenchmarkDotNet.Running.BenchmarkRunnerClean.GetResultsFolderPath(String rootArtifactsFolderPath, BenchmarkRunInfo[] benchmarkRunInfos)
   at BenchmarkDotNet.Running.BenchmarkRunnerClean.Run(BenchmarkRunInfo[] benchmarkRunInfos)
   at BenchmarkDotNet.Running.BenchmarkRunner.RunWithDirtyAssemblyResolveHelper(Type type, IConfig config, String[] args)
   at BenchmarkDotNet.Running.BenchmarkRunner.<>c__DisplayClass0_0`1.<Run>b__0()
   at BenchmarkDotNet.Running.BenchmarkRunner.RunWithExceptionHandling(Func`1 run)
   at BenchmarkDotNet.Running.BenchmarkRunner.Run[T](IConfig config, String[] args)
   at Sub